This is the code for random forest training by Raymond Xu.

 In this edition, the longitude and latitude of the original wildfire dataset is used as inputs.

 Here the temporal sequence is not considered in the data splitting.
 

In [1]:
''' This is Research Project titled ML Algorithms for Alberta Forest Occurence Prediction.
    This is the 8th Engineering Research Project, and is hereby delcared as
                            
                            Project Christopher
    
    Version 1.0 - Long Short-Term Memory Classifier
    Data Source: European Space Agency - ERA5
                 Government of Alberta - Historical Wildfire registry and Fire Weather Indices
                 Natural Resources Canada - Vegetation Classification of Canada
    
    AI Diederik - Hongik Ingan, For the Benefits of All Mankind
'''


' This is Research Project titled ML Algorithms for Alberta Forest Occurence Prediction.\n    This is the 8th Engineering Research Project, and is hereby delcared as\n                            \n                            Project Christopher\n    \n    Version 1.0 - Long Short-Term Memory Classifier\n    Data Source: European Space Agency - ERA5\n                 Government of Alberta - Historical Wildfire registry and Fire Weather Indices\n                 Natural Resources Canada - Vegetation Classification of Canada\n    \n    AI Diederik - Hongik Ingan, For the Benefits of All Mankind\n'

In [2]:
import pandas as pd
import numpy as np # type: ignore
from sklearn.model_selection import train_test_split

# Load wildfire and non-wildfire datasets
def load_downsampled_df():
    main_df = pd.read_csv("D:/Shared drives/MECE 788 - Forest Fire Prediction/04_Preprocessing/Cleanup_and_point_selection/downsampled_df.csv",index_col=0)
    main_df['date'] = pd.to_datetime(main_df['date'])
    return main_df


main_df = load_downsampled_df()

# Create the validation dataframe out of data after 2019
validation_df = main_df[main_df['date'] > pd.Timestamp('2019-01-01')]
test_train_df = main_df[main_df['date'] < pd.Timestamp('2019-01-01')]

def En_test_train_validation_split(validation_df, test_train_df, target_variable='fire', test_proportion=0.33):
    """
    Validation data is obtained by taking all data after a certain time. This is similar to model deployment.
    Train and test data are obtained using a stratified split
    """    
    # Step 1: Sort data by month, then by year within each month
    validation_df['month'] = validation_df['date'].dt.month
    validation_df['year'] = validation_df['date'].dt.year

    split_index = int(len(test_train_df) * (1 - test_proportion))
    test_train_df['month'] = test_train_df['date'].dt.month
    test_train_df['year'] = test_train_df['date'].dt.year

    for df in [validation_df,test_train_df]:
        # Sort by month
        df.sort_values(by='month', inplace=True,ascending=True)
        # Sort by year within each month
        for month in df['month'].unique():
            df.loc[df['month'] == month].sort_values(by='year', inplace=True)
    
    # Split data into training and testing sets
    train_df = test_train_df[test_train_df['date'] < pd.Timestamp('2013-01-01')]
    test_df = test_train_df[test_train_df['date'] > pd.Timestamp('2013-01-01')]

    # Split the test and training
    y_test=test_df['fire']
    y_train=train_df['fire']
    y_validation=validation_df['fire']
    X_test=test_df.drop(columns=['fire'])
    X_train=train_df.drop(columns=['fire'])
    X_validation=validation_df.drop(columns=['fire'])
    
    return X_train, X_test, X_validation, y_train, y_test, y_validation

# Get the splits
X_train, X_test, X_validation, y_train, y_test, y_validation = En_test_train_validation_split(validation_df, test_train_df)




C:\Users\rxu\AppData\Local\Temp\ipykernel_28008\100590291.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_df['month'] = validation_df['date'].dt.month
C:\Users\rxu\AppData\Local\Temp\ipykernel_28008\100590291.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_df['year'] = validation_df['date'].dt.year
C:\Users\rxu\AppData\Local\Temp\ipykernel_28008\100590291.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [3]:
X_train.head()

,date,latitude,longitude,high_vegetation_cover,leaf_area_index_high_vegetation,leaf_area_index_low_vegetation,low_vegetation_cover,slope_of_sub_gridscale_orography,type_of_high_vegetation,type_of_low_vegetation,...,fire_count_past_3Days,fire_count_past_7Days,fire_count_past_10Days,fire_count_past_30Days,24hr_max_temperature_1dayLag,24hr_max_temperature_2dayLag,global_noon_LST_2m_temperature_1dayLag,global_noon_LST_2m_temperature_2dayLag,month,year
104032,2005-03-13,54.50,242.75,0.998321,1.417741,0.000000,0.001556,0.006706,2,0,...,0.0,0.0,0.0,0.0,11.699545,14.356647,4.760975,11.200683,3,2005
104005,2005-03-08,54.25,245.25,0.977950,0.839266,2.777072,0.022050,0.004886,19,1,...,0.0,0.0,0.0,0.0,8.398298,13.631983,1.314445,5.868976,3,2005
104031,2005-03-13,49.50,246.00,0.900371,1.510647,2.051366,0.099629,0.018694,2,2,...,0.0,0.0,0.0,0.0,14.594701,13.136620,1.773049,11.612025,3,2005
104004,2005-03-07,54.50,245.50,0.691179,0.829118,2.786724,0.308822,0.004543,19,1,...,0.0,0.0,0.0,0.0,13.731755,12.660513,6.087775,9.173723,3,2005
104002,2005-03-07,54.25,245.50,0.691179,1.076542,2.458123,0.308822,0.004506,19,1,...,0.0,0.0,0.0,0.0,13.526958,12.282427,5.821715,8.732624,3,2005


In [18]:
## Create Pipeline ##
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline

## Preprocessing ##
# Define features to include #
pass_features = ['leaf_area_index_high_vegetation', 'slope_of_sub_gridscale_orography']
categorical_features = ['type_of_high_vegetation']
numeric_features = ['fire_count_past_3Days','fire_count_past_30Days','DMC','global_noon_LST_2m_temperature','BUI',
                'FWI','latitude','FFMC','global_noon_LST_relative_humidity','24hr_max_temperature',
                'global_noon_LST_2m_temperature_1dayLag','global_noon_LST_2m_temperature_2dayLag',
                'high_vegetation_cover','24hr_max_temperature_1dayLag','low_vegetation_cover',
                '24hr_accumulated_precipitation', 'day_of_the_year']
####

# Define custom preprocessing functions. Put any custom functions in SVM_functions.py also so they are accessible by the ensemble
def extract_day_of_year(X):
    day_of_year = X['date'].dt.dayofyear.to_frame(name='day_of_the_year')
    return day_of_year

# Define numeric and categorical transformer below
date_transformer = ColumnTransformer([('date', FunctionTransformer(extract_day_of_year, validate=False), ['date'])], verbose_feature_names_out=False, remainder='passthrough').set_output(transform='pandas')
scale=ColumnTransformer([('scale_transformer',StandardScaler(),numeric_features)],verbose_feature_names_out=False).set_output(transform='pandas')
cate=ColumnTransformer([('categorical_transformer',OneHotEncoder(sparse_output=False),categorical_features)],verbose_feature_names_out=False).set_output(transform='pandas')
pss=ColumnTransformer([('Pass_transformer','passthrough',pass_features)],verbose_feature_names_out=False).set_output(transform='pandas')

feature_union = FeatureUnion([
    ('numeric', scale),
    ('categorical', cate),
    ('pass', pss)
])


# Final Pipeline
Data_pipeline = Pipeline([
    ('date_of_the_year', date_transformer),
    ('feature_union', feature_union)
])

In [19]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

X_train_processed = Data_pipeline.fit_transform(X_train)
X_test_processed = Data_pipeline.transform(X_test)


Proceed splitting the data

In [6]:
import numpy as np

X_train_lstm= np.resize(X_train_processed,(X_train_processed.shape[0],1,X_train_processed.shape[1]))
X_test_lstm = np.resize(X_test_processed,(X_test_processed.shape[0],1,X_test_processed.shape[1]))

Setup LSTM model that is compatible for hyperparameter tuning

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input

from keras_tuner.tuners import Hyperband
import keras_tuner as kt
import tensorflow.keras.backend as K
from keras.metrics import AUC

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def LSTM_classifier(hp):
    model = Sequential()

    # Define the input shape
    model.add(Input(shape=(1, X_test_processed.shape[1])))

    # Tune the number of LSTM layers
    for i in range(1, 3):  # Adjust the range as needed
        # Add LSTM layer with forget and memory gates
        model.add(LSTM(units=hp.Int(f'units_{i}', min_value=X_test_processed.shape[1], max_value=X_test_processed.shape[1]*5, step=2),
                       activation=hp.Choice(f'activation_{i}', ['relu', 'tanh', 'sigmoid', 'swish', 'linear']),
                       recurrent_activation='sigmoid',  # Use sigmoid for forget and memory gates
                       return_sequences=(i < 2)))  # Return sequences for all layers except the last one

    # Tune whether to use dropout
    if hp.Boolean("dropout"):
        model.add(Dropout(rate=0.20))

    # Add the output layer
    model.add(Dense(1, activation='sigmoid'))

    # Define the optimizer learning rate as a hyperparameter
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.AUC()])

    return model



Hyperband tuning set up below:

In [8]:
tuner = kt.Hyperband(LSTM_classifier,
                     objective=kt.Objective("val_auc", direction="max"),
                     max_epochs=10,
                     factor=3,
                     executions_per_trial=2,
                     overwrite=False,
                     directory='LSTM_Hyperband_tuning',
                     project_name='LSTM_R2')

tuner.search(X_train_lstm, y_train, epochs=10, validation_data=(X_test_lstm, y_test))

Trial 30 Complete [00h 01m 51s]
val_auc: 0.8486031591892242

Best val_auc So Far: 0.8488591611385345
Total elapsed time: 08h 38m 44s


In [9]:
best_classifiers = tuner.get_best_models(num_models=1)
best_classifier=best_classifiers[0]
best_classifier.fit(x=X_train_lstm, y=y_train, batch_size=64, epochs=5, verbose=1, validation_split=0.2)
best_classifier.save('LSTM_selected parameters_RX.h5')

c:\Users\rxu\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\src\tuners\hyperband.py:435: UserWarning: Model 'sequential' had a build config, but the model cannot be built automatically in `build_from_config(config)`. You should implement `def build_from_config(self, config)`, and you might also want to implement the method  that generates the config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the model (i.e. its variables) upon deserialization.
  model.build_from_config(
c:\Users\rxu\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\saving\saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Epoch 1/5
876/876 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - auc: 0.8603 - binary_accuracy: 0.8201 - loss: 0.4026 - val_auc: 0.8465 - val_binary_accuracy: 0.8673 - val_loss: 0.3202
Epoch 2/5
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - auc: 0.8612 - binary_accuracy: 0.8201 - loss: 0.4024 - val_auc: 0.8465 - val_binary_accuracy: 0.8672 - val_loss: 0.3190
Epoch 3/5
876/876 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - auc: 0.8646 - binary_accuracy: 0.8229 - loss: 0.3960 - val_auc: 0.8455 - val_binary_accuracy: 0.8675 - val_loss: 0.3196
Epoch 4/5
876/876 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - auc: 0.8650 - binary_accuracy: 0.8209 - loss: 0.3980 - val_auc: 0.8437 - val_binary_accuracy: 0.8661 - val_loss: 0.3219
Epoch 5/5
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - auc: 0.8599 - binary_accuracy: 0.8206 - loss: 0.4022 - val_auc: 0.8437 - val_binary_accuracy: 0.8666 - val_loss: 0.3243


In [10]:
from sklearn.metrics import f1_score, make_scorer

#best_classifier.summary()
X_validation_processed = Data_pipeline.transform(X_validation)
X_validation_lstm= np.resize(X_validation_processed,(X_validation_processed.shape[0],1,X_validation_processed.shape[1]))

# Assuming predictions is the output of best_classifier.predict(X_validation_processed)
predictions = best_classifier.predict(X_validation_lstm)

# Convert predictions to binary labels (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

# Calculate the F1 score
f1 = make_scorer(f1_score , average='macro')
f1_score_result = f1_score(y_validation, binary_predictions)

print('F1 score on validation data:', f1_score_result)

141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
F1 score on validation data: 0.6072922893006575


In [20]:
def load_full_df():
    main_df = pd.read_csv("D:/Shared drives/MECE 788 - Forest Fire Prediction/Git_final_deliverable_folder/Models/processed_wildfire_ERA5_FWI.csv",index_col=0)
    main_df['date'] = pd.to_datetime(main_df['date'])
    main_df.rename(columns={'latitude_ERA5': 'latitude', 'longitude_ERA5': 'longitude'},inplace=True)
    unnamed_cols = [col for col in main_df.columns if col.startswith('Unnamed:')]
    main_df.drop(columns=unnamed_cols, inplace=True)
    main_df['type_of_high_vegetation'] = main_df['type_of_high_vegetation'].astype(int)
    main_df['type_of_low_vegetation'] = main_df['type_of_low_vegetation'].astype(int)
    return main_df

main_df = load_full_df()

# Create the validation dataframe out of data after 2019
validation_df = main_df[main_df['date'] > pd.Timestamp('2019-01-01')]
test_train_df = main_df[main_df['date'] < pd.Timestamp('2019-01-01')]

# Get the splits
X1_train, X1_test, X1_validation, y1_train, y1_test, y1_validation = En_test_train_validation_split(validation_df, test_train_df)
X1_train.reset_index(drop=True,inplace=True)
X1_test.reset_index(drop=True,inplace=True)
X1_validation.reset_index(drop=True,inplace=True)

X1_validation_processed = Data_pipeline.transform(X1_validation)
X1_validation_lstm= np.resize(X1_validation_processed,(X1_validation_processed.shape[0],1,X1_validation_processed.shape[1]))

C:\Users\rxu\AppData\Local\Temp\ipykernel_28008\100590291.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_df['month'] = validation_df['date'].dt.month
C:\Users\rxu\AppData\Local\Temp\ipykernel_28008\100590291.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_df['year'] = validation_df['date'].dt.year
C:\Users\rxu\AppData\Local\Temp\ipykernel_28008\100590291.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# Use full dataset now

# Assuming predictions is the output of best_classifier.predict(X_validation_processed)
fulldf_predictions = best_classifier.predict(X1_validation_lstm)
np.save('LSTM_pred_proba.npy',fulldf_predictions)


5637/5637 ━━━━━━━━━━━━━━━━━━━━ 5s 889us/step


In [35]:

# Convert predictions to binary labels (0 or 1)
fulldf_predictions = (predictions > 0.8).astype(int)

# Calculate the F1 score
f1 = make_scorer(f1_score , average='macro')
fulldf_f1_score_result = f1_score(y1_validation, fulldf_predictions)

print('F1 score on validation data:', fulldf_f1_score_result)

F1 score on validation data: 0.2027363184079602


In [36]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from sklearn.metrics import roc_curve, auc, confusion_matrix

def plot_roc_curve(y_true, probabilities, point_thresholds=None, point_tprs=None):
    """
    Plots the ROC curve for given probabilities and marks points specified by thresholds or TPRs,
    with legends indicating threshold values and corresponding TPR.
    
    Args:
        y_true (array): True binary labels.
        probabilities (array): Probabilities of the positive class.
        point_thresholds (list): List of thresholds for which points to mark on the ROC curve.
        point_tprs (list): List of TPRs for which points to mark on the ROC curve.
    """
    # Calculate ROC curve points
    fpr, tpr, thresholds = roc_curve(y_true, probabilities)
    roc_auc = auc(fpr, tpr)
    
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')

    # Collect legend information
    legend_handles = [plt.Line2D([0], [0], color='darkorange', lw=2)]
    legend_labels = [f'ROC curve (area = {roc_auc:.2f})']

    # Handle point_thresholds
    if point_thresholds is not None:
        for thresh in point_thresholds:
            idx = np.argmin(np.abs(thresholds - thresh))
            plt.plot(fpr[idx], tpr[idx], 'ro')
            plt.vlines(x=fpr[idx], ymin=0, ymax=tpr[idx], color='grey', linestyle='--')
            plt.hlines(y=tpr[idx], xmin=0, xmax=fpr[idx], color='grey', linestyle='--')
            print(f"Threshold: {thresh:.8f}, TPR: {tpr[idx]:.4f}, FPR: {fpr[idx]:.4f}")
            legend_handles.append(plt.Line2D([0], [0], color='red', marker='o', linestyle=''))
            legend_labels.append(f'Point (Thresh={thresh:.3f}, TPR={tpr[idx]:.3f})')

    # Handle point_tprs
    if point_tprs is not None:
        for t in point_tprs:
            idx = np.argmin(np.abs(tpr - t))
            plt.plot(fpr[idx], tpr[idx], 'bo')
            plt.vlines(x=fpr[idx], ymin=0, ymax=tpr[idx], color='grey', linestyle='--')
            plt.hlines(y=tpr[idx], xmin=0, xmax=fpr[idx], color='grey', linestyle='--')
            print(f"Threshold: {thresholds[idx]:.8f}, TPR: {t:.4f}, FPR: {fpr[idx]:.4f}")
            legend_handles.append(plt.Line2D([0], [0], color='blue', marker='o', linestyle=''))
            legend_labels.append(f'Point (Thresh={thresholds[idx]:.3f}, TPR={t:.3f})')

    # Add legend to plot
    plt.legend(legend_handles, legend_labels, loc="lower right")
    plt.show()

def plot_multiple_ROC(y_true, y_prediction_proba, names):
    """
    Plot ROC curves for multiple predictions on the same plot.

    Parameters:
    - y_true: array-like of shape (n_samples,) - True binary labels.
    - y_prediction_proba: list of array-like predictions from different models, 
                          each of shape (n_samples,).
    - names: list of strings, names corresponding to each prediction in y_prediction_proba.
    
    Each element in y_prediction_proba corresponds to a prediction array from a model,
    and each element in names is the name of the model used in the legend.
    """
    plt.figure(figsize=(10, 8))
    for y_pred, name in zip(y_prediction_proba, names):
        # Compute ROC curve and ROC area for each class
        fpr, tpr, _ = roc_curve(y_true, y_pred)
        roc_auc = auc(fpr, tpr)
        
        # Plot the ROC curve
        plt.plot(fpr, tpr, label=f'{name}: AUC = {roc_auc:.2f}')
    
    plt.plot([0, 1], [0, 1], 'k--')  # Plot the diagonal 45 degree line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()

def get_cm(y_true, y_predict, save_path):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_predict)
    
    # Calculate the percentage of each value
    cm_sum = np.sum(cm)
    cm_percentage = cm / cm_sum * 100

    # Create annotation labels
    # Combine count and percentage in the annotation, formatted for display
    annot_labels = (np.asarray(["{}\n({:.2f}%)".format(count, percentage)
                                for count, percentage in zip(cm.flatten(), cm_percentage.flatten())])
                    .reshape(cm.shape))
    
    # Plotting the heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=annot_labels, fmt='', cmap='Blues', cbar=True)  # Using '' for fmt as we're passing strings in annot_labels
    plt.title("Confusion Matrix")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    
    # Save the figure
    plt.savefig(save_path)
    plt.close()  # Close the plot to free memory if generating multiple plots

## Functions for plotting predictions ##
def add_back_original_features(X_processed,df_original):
    """
    Adds removed features from the original dataframe back to a processed dataframe
    """
    non_overlapping_columns = [col for col in df_original.columns if col not in X_processed.columns]
    df_right_selected = df_original[non_overlapping_columns]
    X_combined = pd.merge(X_processed, df_right_selected, left_index=True, right_index=True, how='left')
    return X_combined

def prepare_df_for_plotting(X,y_pred,df):
    """
    X is the test inputs, y_pred is the resulting model predictions
    df is the original dataframe to add back date, latitude, and longitude columns
    """
    df = add_back_original_features(X,df)
    df['prediction'] = y_pred
    # Classify each prediction
    df['result'] = 'TN'
    df.loc[(df['fire'] == 1) & (df['prediction'] == 1), 'result'] = 'TP'
    df.loc[(df['fire'] == 0) & (df['prediction'] == 1), 'result'] = 'FP'
    df.loc[(df['fire'] == 1) & (df['prediction'] == 0), 'result'] = 'FN'

    return df

def plot_daily_predictions(df,save_dir):
    """
    Plots and saves each plot as a png into save_dir.
    Use prepare_df_for_plotting to get prediction column (true negative, true positive, etc.)
    Parameters:
        df = dataframe with a column that classifies prediction type (TN, TP, FP, FN)
        save_dir = path to directory to save daily images
    """
    os.makedirs(save_dir,exist_ok=True)
    df['date'] = pd.to_datetime(df['date']).dt.date
    # Define the geographic bounds of Alberta
    extent = [-120, -110, 49, 60]  # [west, east, south, north]
    provinces = cfeature.NaturalEarthFeature(
            category='cultural',
            name='admin_1_states_provinces_lines',
            scale='50m',
            facecolor='none',
            edgecolor='black'
        )

    for date in df['date'].unique():
        df_date = df[df['date'] == date]

        fig = plt.figure(figsize=(10, 10))
        # Define the projection and extent
        ax = plt.axes(projection=ccrs.LambertConformal(central_longitude=-115, central_latitude=55))
        ax.set_extent(extent)
        
        # Add geographic features
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        ax.add_feature(cfeature.LAND)
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.LAKES, alpha=0.5)
        ax.add_feature(provinces, linestyle='-', edgecolor='black')

        # Plot points for the given date
        categories = ['TP', 'TN', 'FP', 'FN']
        colors = ['green', 'blue', 'red', 'yellow']
        for category, color in zip(categories, colors):
            df_cat = df_date[df_date['result'] == category]
            plt.scatter(df_cat['longitude'], df_cat['latitude'], c=color, label=category, alpha=0.6, transform=ccrs.PlateCarree())
        
        plt.title(f'Fire Prediction Results for {date}')
        plt.legend(loc='lower left')
        
        # Save the plot
        plt.savefig(os.path.join(save_dir, f'{date}_predictions.png'))
        plt.close(fig)

def plot_incorrect_predictions(df, save_path):
    """
    Plots a heatmap of incorrect predictions over a dataset.
    Use prepare_df_for_plotting to get prediction column (true negative, true positive, etc.)
    Parameters:
        df = dataframe with prediction column that classifies prediction type (TP, TN, FP, FN)
        save_path = path to save the heatmap image
    """
    # Create a DataFrame of all unique latitude and longitude pairs with zero counts
    all_coords = df[['latitude', 'longitude']].drop_duplicates()
    all_coords['counts'] = 0

    # Filter to include only incorrect predictions
    incorrect_df = df[(df['result'] == 'FP') | (df['result'] == 'FN')]
    incorrect_counts = incorrect_df.groupby(['latitude', 'longitude']).size().reset_index(name='counts_fn')

    # Merge the counts back to all coordinates
    agg_df = all_coords.merge(incorrect_counts, on=['latitude', 'longitude'], how='left')
    agg_df['counts'] = agg_df['counts_fn'].fillna(0) + agg_df['counts']
    agg_df.drop(columns=['counts_fn'], inplace=True)

    # Plotting
    fig = plt.figure(figsize=(40, 40))
    ax = plt.axes(projection=ccrs.LambertConformal(central_longitude=-115, central_latitude=55))
    ax.set_extent([-120, -110, 49, 60])

    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAKES, alpha=0.5)
    ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', scale='50m', facecolor='none', edgecolor='black'), linestyle='-')

    norm = plt.Normalize(vmin=agg_df['counts'].min(), vmax=agg_df['counts'].max())
    scatter = ax.scatter(agg_df['longitude'], agg_df['latitude'], c=agg_df['counts'], cmap='Reds', norm=norm, edgecolor='k', linewidth=0.5, alpha=0.7, s=600, transform=ccrs.PlateCarree())
    cbar = plt.colorbar(scatter, shrink=0.5, aspect=5)
    cbar.set_label('Number of Incorrect Predictions', fontsize=40)
    cbar.ax.tick_params(labelsize=30)

    plt.title('Incorrect Predictions Across Alberta', fontsize=40)
    plt.savefig(save_path)
    plt.close(fig)

def plot_false_positives(df, save_path):
    """
    Same as plot_incorrect_predictions except plots false positives rather than incorrect predictions
    Plots a heatmap of incorrect predictions over a dataset.
    Use prepare_df_for_plotting to get prediction column (true negative, true positive, etc.)
    Parameters:
        df = dataframe with prediction column that classifies prediction type (TP, TN, FP, FN)
        save_path = path to save the heatmap image
    """
    # Create a DataFrame of all unique latitude and longitude pairs with zero counts
    all_coords = df[['latitude', 'longitude']].drop_duplicates()
    all_coords['counts'] = 0

    # Filter to include only false positive predictions
    fp_df = df[df['result'] == 'FP']
    fp_counts = fp_df.groupby(['latitude', 'longitude']).size().reset_index(name='counts_fn')

    # Merge the counts back to all coordinates
    agg_df = all_coords.merge(fp_counts, on=['latitude', 'longitude'], how='left')
    agg_df['counts'] = agg_df['counts_fn'].fillna(0) + agg_df['counts']
    agg_df.drop(columns=['counts_fn'], inplace=True)

    # Plotting
    fig = plt.figure(figsize=(40, 40))
    ax = plt.axes(projection=ccrs.LambertConformal(central_longitude=-115, central_latitude=55))
    ax.set_extent([-120, -110, 49, 60])  # Geographic bounds for Alberta

    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAKES, alpha=0.5)
    ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', scale='50m', facecolor='none', edgecolor='black'), linestyle='-')

    norm = plt.Normalize(vmin=agg_df['counts'].min(), vmax=agg_df['counts'].max())
    scatter = ax.scatter(agg_df['longitude'], agg_df['latitude'], c=agg_df['counts'], cmap='Reds', norm=norm, edgecolor='k', linewidth=0.5, alpha=0.7, s=600, transform=ccrs.PlateCarree())
    cbar = plt.colorbar(scatter, shrink=0.5, aspect=5)
    cbar.set_label('Number of False Positives', fontsize=40)
    cbar.ax.tick_params(labelsize=30)

    plt.title('False Positive Predictions Across Alberta', fontsize=40)
    plt.savefig(save_path)
    plt.close(fig)

def plot_false_negatives(df, save_path):
    """
    Variant of plot_incorrect_predictions for looking at false negatives specifically.
    """
    # Create a DataFrame of all unique latitude and longitude pairs
    all_coords = df[['latitude', 'longitude']].drop_duplicates()
    all_coords['counts'] = 0  # Initialize counts to zero

    # Filter to include only false negative predictions
    fn_df = df[df['result'] == 'FN']

    # Group by latitude and longitude, count occurrences
    fn_counts = fn_df.groupby(['latitude', 'longitude']).size().reset_index(name='counts')

    # Merge the counts back to all coordinates
    agg_df = all_coords.merge(fn_counts, on=['latitude', 'longitude'], how='left', suffixes=('', '_fn'))
    agg_df['counts'] = agg_df['counts_fn'].fillna(0) + agg_df['counts']
    agg_df.drop(columns=['counts_fn'], inplace=True)

    # Plotting
    fig = plt.figure(figsize=(40, 40))
    ax = plt.axes(projection=ccrs.LambertConformal(central_longitude=-115, central_latitude=55))
    ax.set_extent([-120, -110, 49, 60])  # Geographic bounds for Alberta

    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAKES, alpha=0.5)
    ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', scale='50m', facecolor='none', edgecolor='black'), linestyle='-')

    norm = plt.Normalize(vmin=agg_df['counts'].min(), vmax=agg_df['counts'].max())
    scatter = ax.scatter(agg_df['longitude'], agg_df['latitude'], c=agg_df['counts'], cmap='Blues', norm=norm, edgecolor='k', linewidth=0.5, alpha=0.7, s=600, transform=ccrs.PlateCarree())
    cbar = plt.colorbar(scatter, shrink=0.5, aspect=5)
    cbar.set_label('Number of False Negatives', fontsize=40)
    cbar.ax.tick_params(labelsize=30)

    plt.title('False Negative Predictions Across Alberta', fontsize=40)
    plt.savefig(save_path)
    plt.close(fig)

## High Level Function for getting all visualizations ##
def generate_visualizations(X_processed,y_predict,y_true,df_original,save_path):
    """
    Generates visualizations for model predictions
    """
    os.makedirs(save_path,exist_ok=True)
    # Add features back to prediction dataset
    df = prepare_df_for_plotting(X_processed,y_predict,df_original)
    
    # Prepare and save plots
    get_cm(y_true,y_predict,os.path.join(save_path,'cm.png'))
    plot_incorrect_predictions(df, os.path.join(save_path,'incorrect_predictions.png'))
    plot_false_positives(df, os.path.join(save_path,'false_positives.png'))
    plot_false_negatives(df, os.path.join(save_path,'false_negatives.png'))
    #plot_daily_predictions(df,os.path.join(save_path,'validation_predictions'))


generate_visualizations(X1_validation,fulldf_predictions,y1_validation,main_df,'D:/Shared drives/MECE 788 - Forest Fire Prediction/05_Analysis/02_LSTM')